# 01. Create COG

Implementation moved to [ns_cogger](./ns_cogger.py). See that file for details.


In [3]:
from ns_configs import RGB_ASSET_CONFIG, MS_ASSET_CONFIG
from ns_cogger import create_cog
from pathlib import Path

force = True
limit = None
input_dir = Path("/home/jamesg/test_data/naturescan")

## RGB COGs

In [4]:
# Get All RGB Tif Files
rgb_tif_files = list(input_dir.rglob("*rgb.tif", case_sensitive=False))

# Limit total (does nothing if limit is None)
rgb_tif_files = rgb_tif_files[0:limit]
print(f"Found {len(rgb_tif_files)} RGB GeoTiff files.")

# Create COG for each
for i, input in enumerate(rgb_tif_files):
    print(f"Processing {input.name} ({i + 1} / {len(rgb_tif_files)}) ...")
    try:
        create_cog(input, input.with_suffix('.cog.tif'), RGB_ASSET_CONFIG, force=force)
    except Exception as e:
        print(f"Error processing: {input.name}")
        print(e)

Found 14 RGB GeoTiff files.
Processing 20241210_SANSSTP014_m3m_50mAGL_ortho_rgb.tif (1 / 14) ...
Successfully converted 20241210_SANSSTP014_m3m_50mAGL_ortho_rgb.tif to 20241210_SANSSTP014_m3m_50mAGL_ortho_rgb.cog.tif.
Processing 20241210_SANSSTP020_m3m_50mAGL_ortho_rgb.tif (2 / 14) ...
Successfully converted 20241210_SANSSTP020_m3m_50mAGL_ortho_rgb.tif to 20241210_SANSSTP020_m3m_50mAGL_ortho_rgb.cog.tif.
Processing 20240812_SANSSTP009_m3m_70mAGL_ortho_rgb.tif (3 / 14) ...
Successfully converted 20240812_SANSSTP009_m3m_70mAGL_ortho_rgb.tif to 20240812_SANSSTP009_m3m_70mAGL_ortho_rgb.cog.tif.
Processing 20241209_SANSSTP010_m3m_50mAGL_ortho_rgb.tif (4 / 14) ...
Successfully converted 20241209_SANSSTP010_m3m_50mAGL_ortho_rgb.tif to 20241209_SANSSTP010_m3m_50mAGL_ortho_rgb.cog.tif.
Processing 20241208_SANSSTP005_m3m_60mAGL_ortho_rgb.tif (5 / 14) ...
Successfully converted 20241208_SANSSTP005_m3m_60mAGL_ortho_rgb.tif to 20241208_SANSSTP005_m3m_60mAGL_ortho_rgb.cog.tif.
Processing 20241208_SA

## MS COGs

In [5]:
# Get All RGB Tif Files
ms_tif_files = list(input_dir.rglob("*ms.tif", case_sensitive=False))

# Limit total (does nothing if limit is None)
ms_tif_files = ms_tif_files[0:limit]
print(f"Found {len(ms_tif_files)} MS GeoTiff files.")

# Create COG for each
for i, input in enumerate(ms_tif_files):
    print(f"Processing {input.name} ({i + 1} / {len(ms_tif_files)}) ...")
    try:
        create_cog(input, input.with_suffix('.cog.tif'), MS_ASSET_CONFIG, force=force)
    except Exception as e:
        print(f"Error processing: {input.name}")
        print(e)

Found 14 MS GeoTiff files.
Processing 20241210_SANSSTP014_m3m_50mAGL_ortho_ms.tif (1 / 14) ...
Successfully converted 20241210_SANSSTP014_m3m_50mAGL_ortho_ms.tif to 20241210_SANSSTP014_m3m_50mAGL_ortho_ms.cog.tif.
Processing 20241210_SANSSTP020_m3m_50mAGL_ortho_ms.tif (2 / 14) ...
Successfully converted 20241210_SANSSTP020_m3m_50mAGL_ortho_ms.tif to 20241210_SANSSTP020_m3m_50mAGL_ortho_ms.cog.tif.
Processing 20240812_SANSSTP009_m3m_70mAGL_ortho_ms.tif (3 / 14) ...
Successfully converted 20240812_SANSSTP009_m3m_70mAGL_ortho_ms.tif to 20240812_SANSSTP009_m3m_70mAGL_ortho_ms.cog.tif.
Processing 20241209_SANSSTP010_m3m_50mAGL_ortho_ms.tif (4 / 14) ...
Successfully converted 20241209_SANSSTP010_m3m_50mAGL_ortho_ms.tif to 20241209_SANSSTP010_m3m_50mAGL_ortho_ms.cog.tif.
Processing 20241208_SANSSTP005_m3m_60mAGL_ortho_ms.tif (5 / 14) ...
Successfully converted 20241208_SANSSTP005_m3m_60mAGL_ortho_ms.tif to 20241208_SANSSTP005_m3m_60mAGL_ortho_ms.cog.tif.
Processing 20241208_SANSSTP006_m3m_110